In [19]:
import pandas as pd
import numpy as np
import os
import PIL
from pathlib import Path
from PIL import Image
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import IPython
from IPython.display import display
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Convolution2D, MaxPooling2D
from keras.callbacks import ReduceLROnPlateau
from keras.applications.inception_v3 import InceptionV3
from keras.utils.np_utils import to_categorical
import tensorflow as tf
        

In [2]:

root_path = Path().resolve()
img = Image.open(f'{root_path}/data/training/aeroplane/2007_000033.jpg')
print(img.size)
# img
maxPixel = 128
imScalled = img.resize((maxPixel,maxPixel))
img_pixel = np.reshape(list(imScalled.getdata()),(maxPixel,maxPixel,3))
img_pixel
print(img_pixel.size)

(500, 366)
49152


In [3]:
maxPixel=128
train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2,shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
valid_datagen = ImageDataGenerator( rescale = 1.0/255. )

train_generator = train_datagen.flow_from_directory(os.path.join(root_path, 'data/training'),  batch_size=200, class_mode = 'binary', target_size = (img.size[0], img.size[1]),classes=['aeroplane','others'])
valid_generator = valid_datagen.flow_from_directory(os.path.join(root_path, 'data/validation'),  batch_size=200, class_mode = 'binary', target_size = (img.size[0], img.size[1]),classes=['aeroplane','others'])

#train_generator = train_datagen.flow_from_directory(os.path.join(root_path, 'data/training'),  batch_size=20, class_mode = 'binary', target_size = (maxPixel, maxPixel),classes=['aeroplane','others'])
#valid_generator = valid_datagen.flow_from_directory(os.path.join(root_path, 'data/validation'),  batch_size=20, class_mode = 'binary', target_size = (maxPixel, maxPixel),classes=['aeroplane','others'])

base_model = InceptionV3(input_shape = (img.size[0], img.size[1], 3), include_top = False, weights = 'imagenet', classes=20)
#base_model = InceptionV3(input_shape = (maxPixel, maxPixel, 3), include_top = False, weights = 'imagenet', classes=20)

for layer in base_model.layers:
    layer.trainable = False


Found 83 images belonging to 2 classes.
Found 723 images belonging to 2 classes.


2022-09-04 08:43:16.518326: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-09-04 08:43:16.551510: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-04 08:43:16.552505: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 980M computeCapability: 5.2
coreClock: 1.1265GHz coreCount: 12 deviceMemorySize: 7.94GiB deviceMemoryBandwidth: 149.31GiB/s
2022-09-04 08:43:16.552606: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:/usr/lib/cuda/include:/usr/lib/cuda/lib64:/usr/lib/cuda/inc

In [4]:
base_model.output

<KerasTensor: shape=(None, 14, 10, 2048) dtype=float32 (created by layer 'mixed10')>

In [5]:
x = Flatten()(base_model.output)
x = Dense(1000, activation='relu')(x)
#x = Dropout(0.2)(x)

# Add a final sigmoid layer with 1 node for classification output
#x = Dense(1000, activation='relu')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

2022-09-04 08:43:18.685117: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1146880000 exceeds 10% of free system memory.
2022-09-04 08:43:19.181434: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1146880000 exceeds 10% of free system memory.
2022-09-04 08:43:19.340334: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1146880000 exceeds 10% of free system memory.


In [6]:
print(Flatten()(model.output))
500*366*3

KerasTensor(type_spec=TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None), name='flatten_1/Reshape:0', description="created by layer 'flatten_1'")


549000

In [7]:
model.output

<KerasTensor: shape=(None, 1000) dtype=float32 (created by layer 'dense')>

In [13]:
# Fit the model
plane_detector = model.fit(
    train_generator, 
    validation_data=valid_generator, 
    steps_per_epoch=1,
#     validation_steps=valid_generator.samples/valid_generator.batch_size,
    epochs=10
)

Epoch 1/10
1/1 [==============================] - 87s 87s/step - loss: 4.5430 - acc: 0.0000e+00 - val_loss: 0.7202 - val_acc: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 86s 86s/step - loss: 4.4537 - acc: 0.0000e+00 - val_loss: 0.6088 - val_acc: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 86s 86s/step - loss: 4.3698 - acc: 0.0000e+00 - val_loss: 0.5791 - val_acc: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 87s 87s/step - loss: 4.3366 - acc: 0.0000e+00 - val_loss: 0.6124 - val_acc: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 86s 86s/step - loss: 4.3221 - acc: 0.0000e+00 - val_loss: 0.6723 - val_acc: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 87s 87s/step - loss: 4.3057 - acc: 0.0000e+00 - val_loss: 0.5145 - val_acc: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 86s 86s/step - loss: 4.2504 - acc: 0.0000e+00 - val_loss: 0.5734 - val_acc: 0.0000e+00
Epoch 8/10
1/1 [=========================

In [41]:
def get_features(
    data_generator: str,
    feature_dimension: float = 1000.,
    sample_size: int = 83,
    batch_size: int = 200
):
    features = np.zeros(shape=(sample_size, feature_dimension))  # Must be equal to the output of the model
    labels = np.zeros(shape=(sample_size))
    
    # Pass data through convolutional base
    i = 0
    for inputs_batch, labels_batch in data_generator:
        features_batch = model.predict(inputs_batch)
        if features_batch.shape[0] != batch_size:
            features[i * batch_size:] = features_batch
            labels[i * batch_size:] = labels_batch
        else:
            features[i * batch_size: (i + 1) * batch_size] = features_batch
            labels[i * batch_size: (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_size:
            break
    
    return features, labels 

In [42]:
# Get train features
train_features, train_labels = get_features(
    data_generator=train_generator,
    feature_dimension=1000,
    sample_size=83,
    batch_size=200
)

In [43]:
# Get validation features
validation_features, validation_labels = get_features(
    data_generator=valid_generator,
    feature_dimension=1000,
    sample_size=723,
    batch_size=200
)

In [35]:
features_validation = np.zeros(shape=(723, 1000))  # Must be equal to the output of the model
labels_validation = np.zeros(shape=(723))

# Pass data through convolutional base
i = 0
for inputs_batch_valid, labels_batch_valid in valid_generator:
    features_batch_validation = model.predict(inputs_batch_valid)
    print(features_batch_validation.shape)
    if features_batch_validation.shape[0] != 200:
        b_size = features_batch_validation.shape[0]
        features_validation[i * 200:] = features_batch_validation
        labels_validation[i * 200:] = labels_batch_valid
    else:
        features_validation[i * 200: (i + 1) * 200] = features_batch_validation
        labels_validation[i * 200: (i + 1) * 200] = labels_batch_valid
    i += 1
    if i * 200 >= 723:
        break


(200, 1000)
(200, 1000)
(200, 1000)
(123, 1000)


### Train SVM

In [ ]:
# Scaling the data
sc = StandardScaler()
X_train = sc.fit_transform(features)
X_test = sc.transform(X_test)